In [3]:
import torch
from torch.nn import Linear, MSELoss, Module
from torch.optim import SGD, Adam
import pandas as pd


## Make a class for Elastic Net

In [32]:
class ElasticNet(Module):
    
    def __init__(self,input_size, alpha=1.0, l1_ratio=0.5, fit_intercept=True):      
        
        # Define the linear regression layer
        super(ElasticNet, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        
        # Define the linear regression layer
        self.linear = Linear(input_size, 1,bias=fit_intercept,device='cpu',dtype=torch.float64)

    def forward(self, x):

        return self.linear(x)

    def objfunc(self, y_pred, y_true):

        mse_loss = MSELoss()(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1)
        l2_reg = torch.norm(self.linear.weight, p=2)

        objective = (1/2) * mse_loss + self.alpha * (
            self.l1_ratio * l1_reg + (1 - self.l1_ratio) * (1/2)*l2_reg**2)

        return objective

    def fit(self, X, y, num_epochs=1000, learning_rate=0.01):

        
        optimizer = Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train() 
            optimizer.zero_grad()
            y_pred = self(X)
            obj_val = self.objfunc(y_pred, y)
            loss = MSELoss()(y_pred, y)
            obj_val.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], MSE: {loss.item()}")

    def predict(self, X):

        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        return self.linear.weight


In [13]:
data = pd.read_csv('https://github.com/dvasiliu/AML/blob/main/Data%20Sets/concrete.csv?raw=true')

In [26]:
y = torch.tensor(data['strength'].values)
x = torch.tensor(data.drop(columns=['strength']).values)

In [33]:
model = ElasticNet(input_size=x.shape[1],alpha=0.01,l1_ratio=0.5)

In [35]:
model.fit(x,y,num_epochs=2000,learning_rate=0.02)

/Users/daniel/hardmetal/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1030])) that is different to the input size (torch.Size([1030, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/2000], MSE: 279.7943570747001
Epoch [200/2000], MSE: 279.27410745628157
Epoch [300/2000], MSE: 279.06874884108856
Epoch [400/2000], MSE: 279.01017863181943
Epoch [500/2000], MSE: 278.9983713401415
Epoch [600/2000], MSE: 278.9965521929798
Epoch [700/2000], MSE: 278.99629486526754
Epoch [800/2000], MSE: 278.99621878104875
Epoch [900/2000], MSE: 278.99615671693607
Epoch [1000/2000], MSE: 278.9960917100273
Epoch [1100/2000], MSE: 278.9960217605396
Epoch [1200/2000], MSE: 278.99594645497126
Epoch [1300/2000], MSE: 278.9958656020063
Epoch [1400/2000], MSE: 278.9957790320001
Epoch [1500/2000], MSE: 286.11687984888306
Epoch [1600/2000], MSE: 278.9962973497253
Epoch [1700/2000], MSE: 278.99555642354767
Epoch [1800/2000], MSE: 278.99548235307304
Epoch [1900/2000], MSE: 278.9954037654596
Epoch [2000/2000], MSE: 278.995320452779


In [11]:
x.shape[1]

8